In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from datetime import datetime

In [ ]:
year = ['2011','2012','2013','2014','2015','2016','2017','2018']
data_ = pd.DataFrame()

acs = pd.read_csv("../data/ACS_2011.csv")
acs['year'] = 2011

pp = pd.read_csv("../data/ACS_2012.csv")
pp['year'] = 2012
acs = acs.append(pp)

pp = pd.read_csv("../data/ACS_2013.csv")
pp['year'] = 2013
acs = acs.append(pp)

pp = pd.read_csv("../data/ACS_2014.csv")
pp['year'] = 2014
acs = acs.append(pp)

pp = pd.read_csv("../data/ACS_2015.csv")
pp['year'] = 2015
acs = acs.append(pp)

pp = pd.read_csv("../data/ACS_2016.csv")
pp['year'] = 2016
acs = acs.append(pp)

pp = pd.read_csv("../data/ACS_2017.csv")
pp['year'] = 2017
acs = acs.append(pp)

pp = pd.read_csv("../data/ACS_2018.csv")
pp['year'] = 2018
data_ = acs.append(pp)

In [ ]:
zir = pd.read_csv("../data/Zip_Zri_MultiFamilyResidenceRental.csv")

In [ ]:
data_['geo_id'] = data_['geo_id'].astype(str).str.zfill(5)
zir['RegionName'] = zir['RegionName'].astype(str).str.zfill(5)

In [ ]:
cols_replace = list(zir.columns)[7:]
cols_keep = list(zir.columns)[:7]
zil_new = zir.melt(id_vars=cols_keep, value_vars = cols_replace)
zil_new['variable']= pd.to_datetime(zil_new['variable'])b

In [ ]:
zil_new['year'] = zil_new['variable'].apply(lambda a: a.year)
zil_new.info()

In [ ]:
zil_yr = zil_new.groupby(['RegionName', 'year']).mean().reset_index()
zil_yr.drop(['SizeRank','RegionID'], inplace=True, axis=1)

In [ ]:
zil_yr = zil_yr.dropna()

zil_yr = zil_yr[zil_yr['year'] > 2010]
zil_yr = zil_yr[zil_yr['year'] < 2019]

In [ ]:
df_final = zil_yr.merge(data_, how='left', left_on = ['RegionName','year'], right_on=['geo_id', 'year'])

In [ ]:
df_final.drop(['pop_5_years_over', 'speak_only_english_at_home', 'speak_spanish_at_home_low_english', 
               'pop_15_and_over', 'pop_never_married', 'pop_now_married', 'pop_separated', 'pop_widowed',
              'pop_divorced', 'geoid','speak_spanish_at_home'], axis=1, inplace=True)

In [ ]:
df_final[df_final['geo_id'].isnull()]

In [ ]:
df_final = df_final[df_final['RegionName']!='11249']
df_final = df_final[df_final['RegionName']!='75033']

df_final[df_final['geo_id'].isnull()]

In [ ]:
null_zip = list(zip(df_final.columns, list(df_final.isnull().sum())))
non_zero = []
col_names = []
for i, j in null_zip:
    if j>0:
        non_zero.append(j)
        col_names.append(i)
list(zip(col_names, non_zero))

In [ ]:
df_final[df_final[col_names].isnull().any(axis=1)][col_names]

df_final_sorted = df_final.sort_values(by=['RegionName', 'year'])

In [ ]:
null_2011 = ['associates_degree', 'bachelors_degree', 
             'high_school_diploma', 
             'less_one_year_college', \
             'masters_degree', 
             'one_year_more_college', 
             'pop_25_years_over']

null_2018 = ['white_including_hispanic', 'black_including_hispanic', 
             'amerindian_including_hispanic', 
             'asian_including_hispanic', 
             'commute_5_9_mins', 'commute_35_39_mins', 
             'commute_40_44_mins', 'commute_60_89_mins', 
             'commute_90_more_mins', 'households_retirement_income',
            'male_60_61','male_62_64']

for i in range(0, df_final_sorted.shape[0]):
    if df_final_sorted['year'].iloc[i] == 2011:
        for j in range(0,len(null_2011)):
            df_final_sorted[null_2011[j]].iloc[i] = df_final_sorted[null_2011[j]].iloc[i+1]
    if df_final_sorted['year'].iloc[i] == 2018:
        for j in range(0,len(null_2018)):
            df_final_sorted[null_2018[j]].iloc[i] = df_final_sorted[null_2018[j]].iloc[i-1]


In [ ]:
df_final = df_final_sorted.loc[:, df_final_sorted.columns != 'do_date']
df_final = df_final.dropna()

In [ ]:
sns.histplot(x = 'value', data = df_final)
plt.show()
df_final = df_final[df_final['value'] < 6000]
sns.histplot(x = 'value', data = df_final)
plt.show()

In [ ]:
# Filter Cityies with less than 100,000 people
zil_city = df_final.merge(zir, on = 'RegionName', how = 'left')
zil_city = zil_city[['RegionName','City','total_pop','year']]
zil_city = zil_city[zil_city['year'] == 2018]
zil_city.drop('year', inplace = True, axis= 1)
zil_city = zil_city.groupby('City').sum('total_pop').sort_values(by = 'total_pop')

zil_filter = zil_city[zil_city['total_pop'] >= 100000].reset_index()
zil_filter = zil_filter['City']

In [ ]:
df_filtered = df_final.merge(zir[['RegionName','City']], on = 'RegionName', how = 'left')

df_filtered = df_filtered[df_cities['City'].isin(zil_filter)]

## Cluster on three features

In [ ]:
df_small_fil = df_filtered[['value', 'total_pop', 'median_income']]
df_small_fil

In [ ]:
scaler = StandardScaler().fit(df_small_fil)
features = scaler.transform(df_small_fil)
df_scal = pd.DataFrame(features, columns = df_small_fil.columns)



In [ ]:
columns = df_small_fil.columns
kmeans = KMeans(n_clusters = 3)
y = kmeans.fit_predict(df_small_fil[columns])
   
df_filtered['Cluster'] = y

## Add CPI data

In [ ]:
year = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

cpi = [251.645, 257.081, 263.050, 270.509, 278.802, 288.233, 297.808, 307.660]
cpi_df = pd.DataFrame()
cpi_df['year'] = year
cpi_df['cpi'] = cpi
cpi_df

In [ ]:
df_addition = df_filtered.merge(cpi_df, how = 'left', on = 'year')

## Add Census business data

## Break data into the different Clusters

In [ ]:
cluster1 = df_addition[df_addition['Cluster'] == 0]
print('Size of Cluster 1', cluster1.shape)

cluster2 = df_addition[df_addition['Cluster'] == 1]
print('Size of Cluster 2', cluster2.shape)

cluster3 = df_addition[df_addition['Cluster'] == 2]
print('Size of Cluster 3', cluster3.shape)

nocluster = df_addition
print('Size of all Data', nocluster.shape)